In [ ]:
using Polyhedra
using CDDLib
h = HalfSpace([1, 0], 1) ∩ HalfSpace([-1, 0], 1) ∩ HalfSpace([0, 1], 1) ∩ HalfSpace([0, -1], 1)
p = polyhedron(h, CDDLib.Library());

In [ ]:
using SetProg
model = Model()
@variable(model, E, Ellipsoid())
cref = @constraint(model, E ⊆ p)

In [ ]:
@objective(model, Max, nth_root(volume(E)))

In [ ]:
using Plots
plot(p, ratio=:equal)
plot!(Sets.Ellipsoid([1 0; 0 1], [0, 0]))